In [1]:
import numpy as np
import math;
import pickle
import pandas as pd
from collections import OrderedDict
import importlib
import time
import timeit

import torch
import torch.nn
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader,TensorDataset
from torchvision.models.resnet import *
from torch.autograd import Variable
from torchvision import transforms

import NNs
from NNs import *
importlib.reload(NNs)
import math
from NNs import ResNetDynamic, FeatureBoostedCNN

import glob
import cv2

from torchsummary import summary
from Preprocessing import *
from Preprocessing import ListsTrainDataset, ListsTestDataset

In [2]:
import sys
print(sys.path)

['/home/dimtsi/Dropbox/UvA/1st Semester/Applied Machine Learning/Project', '/home/dimtsi/anaconda3/envs/torch/lib/python37.zip', '/home/dimtsi/anaconda3/envs/torch/lib/python3.7', '/home/dimtsi/anaconda3/envs/torch/lib/python3.7/lib-dynload', '', '/home/dimtsi/.local/lib/python3.7/site-packages', '/home/dimtsi/anaconda3/envs/torch/lib/python3.7/site-packages', '/home/dimtsi/anaconda3/envs/torch/lib/python3.7/site-packages/IPython/extensions', '/home/dimtsi/.ipython']


## LOAD DATA

In [3]:
original_train_images = pickle.load(open("pkl/extraclassified_padded64.pkl", "rb"))
# train_images = train_images[:1000]
original_labels = pickle.load(open("pkl/train_labels.pkl", "rb"))
kaggle_test_images = pickle.load(open("pkl/test_padded64.pkl", "rb"))
kaggle_test_filenames = pickle.load(open("pkl/test_filenames.pkl", "rb"))

## Load handcrafted features

In [4]:
original_haralick = pickle.load(open("features/extraclassified/train_haralick.pkl", "rb"))
original_moments = pickle.load(open("features/extraclassified/train_moments.pkl", "rb"))
original_sizes = pickle.load(open("features/extraclassified/train_sizes.pkl", "rb"))

kaggle_test_haralick = pickle.load(open("features/extraclassified/test_haralick.pkl", "rb"))
kaggle_test_moments = pickle.load(open("features/extraclassified/test_moments.pkl", "rb"))
kaggle_test_sizes = pickle.load(open("features/extraclassified/test_sizes.pkl", "rb"))

train_handcrafted_features = np.concatenate([original_haralick, original_moments,  original_sizes], axis =1)
kaggle_test_handcrafted_features = np.concatenate([kaggle_test_haralick, kaggle_test_moments,  kaggle_test_sizes], axis =1)

## Split to train test mine

In [5]:
test_set_mine_indexes = pickle.load(open("pkl/test_set_mine_indexes_extraclassified.pkl", "rb"))

train_images = [i for j, i in enumerate(original_train_images) if j not in test_set_mine_indexes]
train_labels = [i for j, i in enumerate(original_labels) if j not in test_set_mine_indexes]
#
test_mine_images = [i for j, i in enumerate(original_train_images) if j in test_set_mine_indexes]
test_mine_labels = [i for j, i in enumerate(original_labels) if j in test_set_mine_indexes]

In [6]:
X_train, y_train = train_images, train_labels
X_val, y_val = test_mine_images, test_mine_labels

## CNN

In [8]:
pretrained = resnet50(pretrained = True)
cnn = ResNetDynamic(pretrained.block, pretrained.layers, num_layers = 2, pretrained_nn = None)
#
cnn_dict = torch.load('models/extraclassified/trained_model90.pt', map_location={"cuda:1": "cuda:0", "cuda:2": "cuda:0"})['state_dict']
cnn.load_state_dict(cnn_dict)
cnn = cnn.eval().cuda()
del(pretrained)
# feature_extractor_cnn = nn.Sequential(*list(cnn.children())[:-1])

In [9]:
mean_norm_test, std_norm_test = calc_means_stds(train_images)

def get_cnn_features(model, x):
    features = ...
    mean_norm_test, std_norm_test = calc_means_stds(train_images)

    test_transforms = transforms. Compose([
        transforms.Grayscale(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[mean_norm_test],
                    std =[std_norm_test])
    ])
    
    total_outputs = torch.Tensor().float()
    total_predicted = torch.Tensor().long()
    
    cnn_dataset = ListsTestDataset(x, transform = test_transforms)
    cnn_loader = torch.utils.data.DataLoader(cnn_dataset, batch_size = 32, shuffle = False)

    predictions = []
    for i, images in enumerate(cnn_loader):
        images = Variable(images, requires_grad=False).cuda()
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        
        total_outputs = torch.cat((total_outputs, outputs.cpu()))
        total_predicted = torch.cat((total_predicted, predicted.cpu().long()))

    return (total_outputs.numpy(), total_predicted.numpy())

In [10]:
cnn_train_predictions = get_cnn_features(cnn, X_train)[1]
# cnn_val_features, cnn_val_predictions = get_cnn_features(cnn, X_val)

RuntimeError: CUDA out of memory. Tried to allocate 4.00 MiB (GPU 0; 3.95 GiB total capacity; 2.51 GiB already allocated; 15.19 MiB free; 637.50 KiB cached)

In [27]:
from sklearn.model_selection import StratifiedKFold

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
norm ={}
kf = StratifiedKFold(n_splits=12, random_state=None, shuffle=True)
for train_indexes, validation_indexes in kf.split(X = train_images, y = train_labels):
    X_train = []
    y_train = []
    X_val = []
    y_val = []
    norm = {}

    handcrafted_train = []
    handcrafted_val = []

    for i in train_indexes:
        X_train.append(train_images[i])
        y_train.append(train_labels[i])
        handcrafted_train.append(train_handcrafted_features[i])
    for j in validation_indexes:
        X_val.append(train_images[j])
        y_val.append(train_labels[j])
        handcrafted_val.append(train_handcrafted_features[i])
    norm['train_norm_mean'], norm['train_norm_std'] = calc_means_stds(X_train)
    break
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
handcrafted_train = scaler.fit_transform(handcrafted_train)
handcrafted_val = scaler.fit_transform(handcrafted_val)

/home/dimtsi/anaconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 7 members, which is too few. The minimum number of members in any class cannot be less than n_splits=12.
  % (min_groups, self.n_splits)), Warning)


In [28]:
# class FeatureBoostedCNN(nn.Module):
    
#     def __init__(self, network, num_extra_feats=0, num_classes=121):
#         super(FeatureBoostedCNN, self).__init__()
#         self.convolutional =  nn.Sequential(*list(network.children())[:-2])
#         self.cnn_final_size =  64* network.block.expansion * 2**(network.num_layers-1)
#         self.flattened_size = self.cnn_final_size + num_extra_feats
#         self.fc1 = nn.Linear(self.flattened_size, self.flattened_size//2)
#         self.fc2 = nn.Linear(self.flattened_size//2, num_classes)


#     def forward(self, x):
#         x1 = self.convolutional(x[0])
#         x1 = torch.cat((x1, x[1]),1)
#         x1 = self.fc1(x1)
#         x1 = self.fc2(x1)
#         return x1    

pretrained = resnet50(pretrained = True)
cnn = ResNetDynamic(pretrained.block, pretrained.layers, num_layers = 2, pretrained_nn = None)

# cnn.load_state_dict(torch.load('best_new.pt')['state_dict'])
# feature_extractor_cnn = nn.Sequential(*list(cnn.children())[:-2])
# feature_extractor_cnn
num_handcrafted = handcrafted_train.shape[1]
ensemble_nn = FeatureBoostedCNN(cnn, num_handcrafted)
ensemble_nn

TypeError: super(type, obj): obj must be an instance or subtype of type

In [ ]:

train_dataset, val_dataset = create_train_val_datasets(X_train, y_train,
                                                       X_val, y_val, 
                                                       norm_params = norm,
                                                       train_features = handcrafted_train,
                                                       val_features = handcrafted_val
                                                       )
# train_sampler = ImbalancedDatasetSampler(train_dataset)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 32,
    shuffle = True, num_workers=4)

val_loader = torch.utils.data.DataLoader(val_dataset,
                            batch_size = 32, shuffle = False)

In [ ]:
train_dataset.features

In [ ]:
train_and_validate_with_features(ensemble_nn, train_loader, val_loader, num_epochs=100)

In [ ]:
handcrafted_val.shape